In [1]:
mutable struct Cave
    name::String
    issmall::Bool
    connections::Set{Any}
end

function cave_system_from_file(filename)
    cave_system = Dict([])
    for line in readlines(filename)
        caves = split(line, '-')
        cave1 = Cave(caves[1], all(islowercase.(collect(caves[1]))), Set([caves[2]]))
        cave2 = Cave(caves[2], all(islowercase.(collect(caves[2]))), Set([caves[1]]))
        for c in [cave1, cave2]
            c.connections = filter(cv->(cv!=="start"), c.connections)
            if !(haskey(cave_system, c.name))
                cave_system[c.name] = c
            else
                push!(cave_system[c.name].connections, c.connections...)
            end
        end
    end
    return cave_system
end

function cave_traverse_part1(cave_name, cave_system::Dict{}, parents)
    cave = cave_system[cave_name]
    if cave_name == "end"
        return 1
    else
        new_parents = vcat(parents, [cave_name])
        descend_to = filter(x->(!cave_system[x].issmall | (x ∉ parents)), cave.connections)
        # Sum that works on empty iterables
        return reduce(+, [cave_traverse_part1(c, cave_system, new_parents) for c in descend_to], init=0)
    end
end

function solve_part1(filename)
    cave_sys = cave_system_from_file(filename)
    return cave_traverse_part1("start", cave_sys, [])
end

println("Part 1 small test: $(solve_part1("small_test.txt"))")
println("Part 1 medium test: $(solve_part1("medium_test.txt"))")
println("Part 1 big test: $(solve_part1("big_test.txt"))")
println("Part 1 solution: $(solve_part1("input.txt"))")

Part 1 small test: 10
Part 1 medium test: 19
Part 1 big test: 226
Part 1 solution: 5958


In [2]:
function cave_traverse_part2(cave_name, cave_system::Dict{}, parents)
    cave = cave_system[cave_name]
    if cave_name == "end"
        return 1
    else
        new_parents = vcat(parents, [cave_name])
        descend_to = filter(x->(!cave_system[x].issmall | (x ∉ parents)), cave.connections)
        extra_small_connections = filter(x->(cave_system[x].issmall & (x ∉ ["start", "end"])), cave.connections)

        num_occurrences = []
        for c in new_parents 
            if (cave_system[c].issmall & (c ∉ ["start", "end"]))
                append!(num_occurrences, count(x->(x==c), new_parents))
                if num_occurrences[end] > 1
                    break # Short circuit for long parent lists
                end
            end
        end
        
        if all(num_occurrences .<= 1)
            if length(extra_small_connections) > 0
                union!(descend_to, Set(extra_small_connections))
            end
        end
        # Sum that works on empty iterables
        return reduce(+, [cave_traverse_part2(c, cave_system, new_parents) for c in descend_to], init=0)
    end
end

function solve_part2(filename)
    cave_sys = cave_system_from_file(filename)
    return cave_traverse_part2("start", cave_sys, [])
end

println("Part 2 small test: $(solve_part2("small_test.txt"))")
println("Part 2 medium test: $(solve_part2("medium_test.txt"))")
println("Part 2 big test: $(solve_part2("big_test.txt"))")
println("Part 2 solution: $(solve_part2("input.txt"))")

Part 2 small test: 36
Part 2 medium test: 103


Part 2 big test: 3509


Part 2 solution: 150426
